In [13]:
# import libraries 
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [14]:
# Function to extract Product Title
def get_title(soup):
    try:
        # Try to find an HTML element with the id "productTitle" using the find method of the BeautifulSoup object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Extract the text of the title element and assign it to a variable
        title_value = title.text
        
        # Remove leading and trailing whitespace from the title text and assign the result to a variable
        title_string = title_value.strip()
    except AttributeError:
        # If the title element can't be found or doesn't have a text attribute, assign an empty string to the title variable
        title_string = ""
    
    # Return the title string
    return title_string

# Function to extract Product Price
def get_price(soup):
    try:
        # Try to find an HTML element with the id "price_inside_buybox" using the find method of the BeautifulSoup object
        price = soup.find("span", attrs={'id':'price_inside_buybox'}).text
        
    except:
        # If the above step raises an exception, try to find an HTML element with the class "a-offscreen"
        price = soup.find("span", attrs={'class':'a-offscreen'}).text

    # Return the price (or None if no price was found)
    return price



# Function to extract Product Rating
def get_rating(soup):
    try:
        # Try to find an HTML element with the class "a-icon a-icon-star a-star-4-5" using the find method of the BeautifulSoup object
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).text
        
    except:
        # If the above step raises an exception, try to find an HTML element with the same class attribute as in the try block
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).text

    # Return the rating (or None if no rating was found)
    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        # Try to find an HTML element with the id "acrCustomerReviewText" using the find method of the BeautifulSoup object
        # Extract the text of the element and strip whitespace from the resulting string
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        # If the above step raises an exception, set review_count to an empty string
        review_count = ""

    # Return the review count (or an empty string if no review count was found)
    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        # Try to find an HTML element with the id "availability" using the find method of the BeautifulSoup object
        # Then find the first <span> element within that element and extract the text, removing any leading or trailing whitespace characters
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        # If the above step raises an exception, set available to "Not Available"
        available = "Not Available"

    # Return the availability status (or "Not Available" if no availability status was found)
    return available


In [18]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    # webpage URL
    URL = "https://www.amazon.com/s?k=laptop&crid=GYZPZZ1UDJU7&sprefix=laptop%2Caps%2C314&ref=nb_sb_noss_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazonweb_data.csv", header=True, index=False)

In [19]:
amazon_df

,title,price,rating,reviews,availability
0,"Lenovo 15.6"" IdeaPad 1 Laptop, 20GB Memory, 1T...",$398.74,4.5 out of 5 stars,62 ratings,Not Available
1,"HP 2022 Newest Pavilion 15.6"" HD Laptop, Intel...",$469.00,4.3 out of 5 stars,207 ratings,Not Available
2,"SGIN Laptop 15.6 Inch, 4GB DDR4 128GB SSD Wind...",$999.98,4.4 out of 5 stars,336 ratings,Not Available
3,"Lenovo 2022 Newest Ideapad 3 Laptop, 15.6"" HD ...",$379.00,4.4 out of 5 stars,"2,294 ratings",Not Available
4,"2021 HP Stream 14"" HD SVA Laptop Computer, Int...",$289.00,,891 ratings,Not Available
5,"HP 2022 Newest Pavilion 15.6"" HD Laptop, Intel...",$469.00,4.3 out of 5 stars,207 ratings,Not Available
6,"2022 Newest Lenovo IdeaPad 3i 15.6"" FHD Laptop...",$499.99,4.4 out of 5 stars,107 ratings,Not Available
7,"Acer 2023 Newest Aspire 5 Slim Laptop, 15.6"" F...",$479.99,4.3 out of 5 stars,29 ratings,Not Available
8,"HP 2022 New 15 Laptop, 15.6"" HD LED Display, I...",$479.00,4.3 out of 5 stars,568 ratings,Not Available
9,2022 HP Premium 14-inch HD Thin and Light Lapt...,$369.00,,332 ratings,Not Available


In [17]:
d

{'title': ['HP Pavilion Laptop (2022 Model), 15.6" HD Display, Intel Celeron Quad-Core Processor, 16GB DDR4 RAM, 1TB SSD, Online Conferencing, Webcam, HDMI, WiFi, Bluetooth, Windows 11',
  '2022 HP Premium 14-inch HD Thin and Light Laptop, Intel Quad-Core Processor, 16GB RAM, 64GB Storage, Long Battery Life, Webcam, Bluetooth, HDMI, Wi-Fi, Rose Gold, Windows 11 + 1 Year Microsoft 365',
  'SGIN Laptop 15.6 Inch, 4GB DDR4 128GB SSD Windows 11 Laptops with Intel Celeron N4020C(up to 2.8 GHz), Intel UHD Graphics 600, Mini HDMI, WiFi, Webcam, USB3.0, Bluetooth 4.2',
  'Lenovo 2022 Newest Ideapad 3 Laptop, 15.6" HD Touchscreen, 11th Gen Intel Core i3-1115G4 Processor, 8GB DDR4 RAM, 256GB PCIe NVMe SSD, HDMI, Webcam, Wi-Fi 5, Bluetooth, Windows 11 Home, Almond',
  '2021 HP Stream 14" HD SVA Laptop Computer, Intel Celeron N4000 Processor, 4GB RAM, 64GB eMMC Flash Memory, Webcam, 1-Year Office, Intel UHD Graphics 600, Win 10S, Rose Pink, 32GB SnowBell USB Card',
  'HP 2022 Newest Pavilion 15.6"